# pROJECT

In [3]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 45.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 52.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 42.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 56.5 MB/s eta 0:00:00:00:01

In [5]:
!pip install scikit-image


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 46.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.6 MB/s eta 0:00:00


In [6]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from skimage.transform import resize



In [7]:
# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Create TensorFlow dataset from numpy arrays
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Define a function to resize images
def resize_image(image, label):
    resized_image = tf.image.resize(image[..., tf.newaxis], (64, 64))  # Resize to 64x64
    return resized_image[:, :, 0], label

# Map the resizing function to the datasets
train_dataset_resized = train_dataset.map(resize_image)
test_dataset_resized = test_dataset.map(resize_image)

In [4]:
# Map the resizing function to the datasets
train_dataset_resized = train_dataset.map(resize_image)
test_dataset_resized = test_dataset.map(resize_image)

In [5]:
# Set batch size
batch_size = 16

# Shuffle and batch the datasets
train_dataset_resized = train_dataset_resized.shuffle(buffer_size=10000).batch(batch_size)
test_dataset_resized = test_dataset_resized.batch(batch_size)

In [6]:
from tensorflow.keras.applications import MobileNet

# Create the MobileNet model with pre-trained weights
base_model = MobileNet(weights=None, input_shape=(64, 64, 1), include_top=False)


In [7]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# Add a new output layer
x = base_model.output
x = Flatten()(x)
x = Dense(10, activation='softmax')(x)

# Define the model with new output layer
model = Model(inputs=base_model.input, outputs=x)


In [8]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with the data generators
model.fit(train_dataset_resized, epochs=10, validation_data=test_dataset_resized)


Epoch 1/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1123s 296ms/step - accuracy: 0.7937 - loss: 0.7045 - val_accuracy: 0.9734 - val_loss: 0.0899
Epoch 2/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1096s 292ms/step - accuracy: 0.9664 - loss: 0.1371 - val_accuracy: 0.9662 - val_loss: 0.1480
Epoch 3/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1125s 300ms/step - accuracy: 0.9736 - loss: 0.1021 - val_accuracy: 0.9863 - val_loss: 0.0458
Epoch 4/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1181s 315ms/step - accuracy: 0.9816 - loss: 0.0678 - val_accuracy: 0.9822 - val_loss: 0.0611
Epoch 5/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1210s 323ms/step - accuracy: 0.9862 - loss: 0.0505 - val_accuracy: 0.9889 - val_loss: 0.0392
Epoch 6/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1217s 321ms/step - accuracy: 0.9885 - loss: 0.0427 - val_accuracy: 0.9877 - val_loss: 0.0452
Epoch 7/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1204s 321ms/step - accuracy: 0.9903 - loss: 0.0337 - val_accuracy: 0.9890 - val_loss: 0.0344
Epoch 8/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1167s 311ms/s